In [1]:
import sys
from os.path import join,expanduser
user_home_dir = expanduser('~')

#point to the correct src path if this doesnt work
sys.path.insert(0,join(user_home_dir,'efs_ecco/ameza/ESS25-Team_SHERLOCCS/src'))

from src import * 
datadir = lambda x: "/efs_ecco/ameza/ESS25-Team_SHERLOCCS/notebooks/property_time_series/data/" + x
maskdir = lambda x: "/efs_ecco/ameza/ESS25-Team_SHERLOCCS/masks/" + x

In [2]:
from dask.distributed import Client

#  connec to existing LocalCluster
# the port number will be different!
client = Client("tcp://127.0.0.1:38937")
# client = Client()
client.ncores
client.restart()
client

<Client: 'tcp://127.0.0.1:41551' processes=0 threads=0, memory=0 B>

In [3]:
## Set top-level file directory for the ECCO NetCDF files
## =================================================================

## currently set to ~/Downloads/ECCO_V4r4_PODAAC, 
## the default if ecco_podaac_download was used to download dataset granules
ECCO_dir = join('/efs_ecco','ameza')

# for access_mode = 's3_open_fsspec', need to specify the root directory 
# containing the jsons
jsons_root_dir = join('/efs_ecco','mzz-jsons')

access_mode = 's3_open_fsspec'

## access datasets needed for this tutorial

ShortNames_list = ["ECCO_L4_GEOMETRY_05DEG_V4R4",\
                   "ECCO_L4_OCEAN_VEL_05DEG_MONTHLY_V4R4",\
                   "ECCO_L4_BOLUS_05DEG_MONTHLY_V4R4",\
                    "ECCO_L4_SSH_05DEG_MONTHLY_V4R4"]
StartDate = '1991-01'
EndDate = '2018-12'
ds_dict = ea.ecco_podaac_to_xrdataset(ShortNames_list,\
                                            StartDate=StartDate,EndDate=EndDate,\
                                            mode=access_mode,\
                                            download_root_dir=ECCO_dir,\
                                            max_avail_frac=0.5,\
                                            jsons_root_dir=jsons_root_dir)

In [4]:
ds_3D_Vel = ds_dict["ECCO_L4_OCEAN_VEL_05DEG_MONTHLY_V4R4"]
ds_3D_Bol = ds_dict["ECCO_L4_BOLUS_05DEG_MONTHLY_V4R4"]
ds_SSH  = ds_dict["ECCO_L4_SSH_05DEG_MONTHLY_V4R4"]
ds_GEOM = ds_dict["ECCO_L4_GEOMETRY_05DEG_V4R4"]

ccc_mask = xr.open_dataset(maskdir("CCC_mask_Anthony.nc"))

In [5]:
from src import regular_vertical_average, generate_averaged_regular_velocity_dataset, select_regular_california

In [6]:
ds_SSH = select_regular_california(ds_SSH["SSH"]).compute()
ds_SSH.to_netcdf(datadir("SSH.nc"))

In [8]:
#calculate average surface layer velocities from 0 to 50 meters depth
ds_surf = generate_averaged_regular_velocity_dataset(ds_3D_Vel, ds_3D_Bol, 0, -50)
ds_surf = select_regular_california(ds_surf).compute()
ds_surf.to_netcdf(datadir("mixed_layer_velocities.nc"))

In [9]:
#calculate average surface layer velocities from 100 to 210 meters depth 
ds_subsurf = generate_averaged_regular_velocity_dataset(ds_3D_Vel, ds_3D_Bol, -100, -210)
ds_subsurf = select_regular_california(ds_subsurf).compute()
ds_subsurf.to_netcdf(datadir("subsurface_layer_velocities.nc"))